# **Data Encryption Standard (DES)** [Prepared By: Jevin Evans and Muhammad Ismail]
---

The **Data Encryption Standard (DES)** is a symmetric encryption scheme that uses the Feistel cipher scheme and four basic operations: exclusive-OR, permutation, substitution, and circular shift.

The following is an in-depth look at the process and features of **DES** to give you a better understanding of the algorithm.

##**Instructions** 
Run in order by hitting the play button next to each code box, or in the <u>menu bar</u>, select **Runtime->Run All** to run the whole lab.

In [2]:
#@title Example for DES
#@markdown For this lab, use the given plaintext and key or choose your own to see how it will be encrypted and decrypted in DES.

PLAINTEXT = 'DES is strong but not complicated' #@param {type: "string"}
KEY = 'CSC4575/5575'  #@param {type: "string"}
#@markdown ---

print(f"Plaintext Message: {PLAINTEXT}\nEncryption Key: {KEY}")

Plaintext Message: DES is strong but not complicated
Encryption Key: CSC4575/5575


## Definitions

Some things to know:
- A **permutation** is a process of shuffling or rearranging data. In DES, permutation is used to shuffle bits given a specific order.

In [3]:
def permute(var:str, order:tuple):
  try:
    '''If var is in binary returns binary'''
    return "".join([var[x-1] for x in order])
  except:
    '''If var is in hex returns hex'''
    var = bin(int(var,16))[2:]
    return hex(int("".join([var[x-1] for x in order]),2))[2:]

# Example
ex1 = "0abcdef"
exOrder = [1,5,6,3,7,2,4]

print(f"Original: {ex1}")
print(f"Pemutation Order: {exOrder}")
print(f"Permutated: {permute(ex1, exOrder)}")

Original: 0abcdef
Pemutation Order: [1, 5, 6, 3, 7, 2, 4]
Permutated: 0debfac


# Key Schedule Generation
---
DES has a key length of 64-bits. The key is first permuted to remove every 8th bit, which can be used as parity bits or discarded, resulting in a 56-bit key. The 56-bit key goes through a **Key Schedule Generation** that produces 16 48-bit subkeys for encryption and decryption. 


## Validation
Before the subkey generation process can fully start, the keys must be validated to the 64-bit size. Users can supply smaller or greater keys than the required size, which requires a method to fit the key to the correct size. In this example, the key is in hex. When the key is smaller than the required size, '0' bits are appended to the end. When the key is too large, the sha256 hash digest is used, and the first 64-bits are kept. 

In [4]:
from hashlib import sha256

def validateKey(key:str):
  try:
    ''' If the key is already in hex '''
    new_key = hex(int(key, 16))[2:18]
    while len(new_key) != 16:
      new_key += '0'
    return new_key
  except ValueError:
    ''' If key in ASCII '''
    return sha256(key.encode()).hexdigest()[0:16]

print(f"Original Key:  {KEY}\nValidated Key (Hex): {validateKey(KEY)}\nValidated Key (Bin): {bin(int(validateKey(KEY),16))[2:]}")

Original Key:  CSC4575/5575
Validated Key (Hex): da458ae1accdb686
Validated Key (Bin): 1101101001000101100010101110000110101100110011011011011010000110


## Subkey Generation
After the key is validated, the subkey generation can begin. The first thing that is done is a permutation using **Permutated Choice 1** on the key. This permutation drops every 8th bit, which can be used for parity or discarded. This drop of bits turns the 64-bit key into a 56-bit key.

In [5]:
# Permutated Choice 1 - Permutation used first on Key (56-bit key)
pc1Order = (57, 49, 41, 33, 25, 17,  9, 
             1, 58, 50, 42, 34, 26, 18, 
            10,  2, 59, 51, 43, 35, 27, 
            19, 11,  3, 60, 52, 44, 36,
            63, 55, 47, 39, 31, 23, 15,
             7, 62, 54, 46, 38, 30, 22,
            14,  6, 61, 53, 45, 37, 29,
            21, 13,  5, 28, 20, 12,  4)

Using the 56-bit key, we will generate 16 48-bit subkeys. The 56-bit key is split into left and right blocks of 28-bits. For 16 rounds, these two blocks are:
1. **Circular Left Shifted** based on the **Shift Round Schedule**
1. Concatenated left+right
1. Permutated using the **Permutation Choice 2** (**pc2Order**), turning the 56-bit key to 48-bits
1. Added to the list of subkeys
1. The process is repeated using the shift halves.

In [6]:
# Permutated Choice 2 - Permutation used for Subkeys (48-bit keys)
pc2Order =  (14, 17, 11, 24,  1,  5,
              3, 28, 15,  6, 21, 10,
             23, 19, 12,  4, 26,  8,
             16,  7, 27, 20, 13,  2,
             41, 52, 31, 37, 47, 55,
             30, 40, 51, 45, 33, 48,
             44, 49, 39, 56, 34, 53, 
             46, 42, 50, 36, 29, 32)

# Order used to Left Circular Shift the Subkeys in key generation
ShiftRounds = (1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1)

def circularLeftShift(var:str, shift:int):
  return var[shift:]+var[0:shift]

Putting all these steps together, the following shows the **Key Schedule Generation** for the previously supplied key:

In [7]:
def keyScheduleGeneration(key:str):
  # Validate Key
  key = validateKey(key)

  # Permute with Permutation Choice 1 
  key = (bin(int('1'+key,16))[3:]).zfill(4) # Converts Hex to Bin
  key = permute(key, pc1Order)

  #Split Keys into 28-bit halves
  LK = key[:28]
  RK = key[28:]

  # Create Subkey List
  subKeys = list()

  # Generate 16 subkeys for each round of DES
  for shift in ShiftRounds:
    # Left Circular Shift key halves
    LK = circularLeftShift(LK, shift)
    RK = circularLeftShift(RK, shift)

    # Concatentate subkey and permutate using the Permutated Choice 2
    newSubKey = permute(LK+RK, pc2Order)
    subKeys.append(newSubKey)

  return subKeys

# Example of Sub Keys
print(f"Original Key:  {KEY}\nValidated Key (Hex): {validateKey(KEY)}\nValidated Key (Bin): {bin(int(validateKey(KEY),16))[2:]}")
print(f"\nSubkeys (48-bits)\n------------------")
for rnd, tempKey in enumerate(keyScheduleGeneration(KEY), start=1):
  print(f"Round {rnd:2d} Subkey: {tempKey}")

Original Key:  CSC4575/5575
Validated Key (Hex): da458ae1accdb686
Validated Key (Bin): 1101101001000101100010101110000110101100110011011011011010000110

Subkeys (48-bits)
------------------
Round  1 Subkey: 110011111001011100010101000111001010011100110110
Round  2 Subkey: 101110110100010100101011100101000101000110101101
Round  3 Subkey: 111010010000001110111001010000100011101011100101
Round  4 Subkey: 100101011001101010011101111100101010100110111001
Round  5 Subkey: 000101110011101011010010001000110001111100011011
Round  6 Subkey: 001111100111110011100100010111110011000100110010
Round  7 Subkey: 110110100110010101001100011001010100100101101100
Round  8 Subkey: 010010001100011100111101010000001011100011011110
Round  9 Subkey: 111111111110010010100000110110001010010101011001
Round 10 Subkey: 110110101000011110101010101010111111011000001000
Round 11 Subkey: 111110001001001000011111011110000111011100100010
Round 12 Subkey: 001001011001101001001110101111000100100000101110
Round 13 Subkey: 0

# 64 Bit Blocks
---
In DES, plaintext messages are encrypted in blocks of size 64-bits. Messages are usually more significant and must be divided into many blocks. DES will convert the ASCII message into binary blocks and pad the last block with zeros if needed. Once the blocks are created, each one will go through the encryption/decryption process.

In [8]:
from math import ceil

# Converts ASCII to Binary
def ascii2bin(var):
  return "".join([bin(ord(x))[2:].zfill(8) for x in var])

# Converts Binary to ASCII
def bin2ascii(var):
  var = [var[i:i+8] for i in range(0, len(var), 8)]
  return ''.join([chr(int(char,2)) for char in var])

def create64Blocks(plaintext):
  # Convert ASCII string to Binary string
  plaintext = ascii2bin(plaintext)
  blocks = list()

  # Add Padding if needed
  if len(plaintext) % 64 != 0:
    plaintext += '0'*(64 - len(plaintext) % 64)
    
  # Determine the total number of blocks
  num_of_blocks = ceil(len(plaintext)/64)

  # Splits the plaintext into 64-bit blocks
  for x in range(num_of_blocks):
    block = ""
    for y in range(64):
      block += plaintext[x*64+y]
    blocks.append(block)
  
  return blocks

testBlocks = create64Blocks(PLAINTEXT)
for index, block in enumerate(testBlocks, start=1):
  print(f"Block {index} ({len(block)}-bit): {bin2ascii(block)}\t{block}")

Block 1 (64-bit): DES is s	0100010001000101010100110010000001101001011100110010000001110011
Block 2 (64-bit): trong bu	0111010001110010011011110110111001100111001000000110001001110101
Block 3 (64-bit): t not co	0111010000100000011011100110111101110100001000000110001101101111
Block 4 (64-bit): mplicate	0110110101110000011011000110100101100011011000010111010001100101
Block 5 (64-bit): d       	0110010000000000000000000000000000000000000000000000000000000000


# Feistel Function
---
The **Feistel function** is a process that performs four operations on a 32-bit block during a DES round. In order, these operations are:
1. **Expansion**
1. **XOR with key**
1. **Substitution with Sbox**
1. **Permutation**

The Feistel function is responsible for adding the subkeys to the blocks during encryption and decryption. The following explains in detail each step.




## Expansion
**Expansion** uses the **Data Expansion** to expand the 32-bit block into a 48-bit block. Expansion is needed so that the key and block with be the same size.

In [9]:
# Order used to expand the bits from 32 to 48
DataExpansion = (32,  1,  2,  3,  4,  5, 
                  4,  5,  6,  7,  8,  9, 
                  8,  9, 10, 11, 12, 13, 
                 12, 13, 14, 15, 16, 17, 
                 16, 17, 18, 19, 20, 21, 
                 20, 21, 22, 23, 24, 25, 
                 24, 25, 26, 27, 28, 29, 
                 28, 29, 30, 31, 32,  1)


print("Left Half of Block 1:", testBlocks[0][:32], len(testBlocks[0][:32]),"bits")

# Expansion of the Block
eBlock = permute(testBlocks[0][:32], DataExpansion)
print("Expanded Left Half of Block 1", eBlock, len(eBlock), "bits")

Left Half of Block 1: 01000100010001010101001100100000 32 bits
Expanded Left Half of Block 1 001000001000001000001010101010100110100100000000 48 bits


## XOR with Key
The block has been expanded to 48-bits, and next is **XOR**'d with the subkey for the coinciding round key.

In [10]:
# XOR's two int numbers and pads front zeros
def XOR(var1:str, var2:str, zeros:int):
  return bin((int(var1, 2)^int(var2,2)))[2:].zfill(zeros)

# XOR Example
firstSubKey = keyScheduleGeneration(KEY)[0]
print(f"First SubKey:\t{firstSubKey}\nExpanded Block: {eBlock}")
print("\t\t"+"-"*48)
xorTemp = XOR(eBlock,firstSubKey, 48)
print(f"XOR Result:\t{xorTemp}")

First SubKey:	110011111001011100010101000111001010011100110110
Expanded Block: 001000001000001000001010101010100110100100000000
		------------------------------------------------
XOR Result:	111011110001010100011111101101101100111000110110


## Substitution w/ Sboxs

**Substitution** in **DES** uses secret boxes (s-boxes) to replace values in the current block with others, which generates confusion, turning the 48-bit output back to 32-bits. 

The 48-bit block can be divided into eight 6-bit chunks. A corresponding s-box matrix (4x16) uses the chunk to pick the replacement value for each chunk.

In [11]:
# Secret Box 1 - 8 for Feistel Function
S1 = (14,  4, 13, 1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9, 0,  7, 
       0, 15,  7, 4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5, 3,  8, 
       4,  1, 14, 8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10, 5,  0, 
      15, 12,  8, 2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0, 6, 13)

S2 = (15,  1,  8, 14,  6, 11,  3,  4,  9, 7,  2, 13, 12, 0,  5, 10, 
       3, 13,  4,  7, 15,  2,  8, 14, 12, 0,  1, 10,  6, 9, 11,  5, 
       0, 14,  7, 11, 10,  4, 13,  1,  5, 8, 12,  6,  9, 3,  2, 15, 
      13,  8, 10,  1,  3, 15,  4,  2, 11, 6,  7, 12,  0, 5, 14,  9)

S3 = (10,  0,  9, 14, 6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8, 
      13,  7,  0,  9, 3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1, 
      13,  6,  4,  9, 8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7, 
       1, 10, 13,  0, 6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12)

S4 = ( 7, 13, 14, 3,  0,  6,  9, 10,  1, 2, 8,  5, 11, 12,  4, 15, 
      13,  8, 11, 5,  6, 15,  0,  3,  4, 7, 2, 12,  1, 10, 14,  9, 
      10,  6,  9, 0, 12, 11,  7, 13, 15, 1, 3, 14,  5,  2,  8,  4, 
       3, 15,  0, 6, 10,  1, 13,  8,  9, 4, 5, 11, 12,  7,  2, 14)

S5 = ( 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13, 0, 14,  9, 
      14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3, 9,  8,  6, 
       4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6, 3,  0, 14, 
      11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10, 4,  5, 3)

S6 = (12,  1, 10, 15, 9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11, 
      10, 15,  4,  2, 7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8, 
       9, 14, 15,  5, 2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6, 
       4,  3,  2, 12, 9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13)

S7 = ( 4, 11,  2, 14, 15, 0,  8, 13,  3, 12, 9,  7,  5, 10, 6,  1, 
      13,  0, 11,  7,  4, 9,  1, 10, 14,  3, 5, 12,  2, 15, 8,  6, 
       1,  4, 11, 13, 12, 3,  7, 14, 10, 15, 6,  8,  0,  5, 9,  2, 
       6, 11, 13,  8,  1, 4, 10,  7,  9,  5, 0, 15, 14,  2, 3, 12)

S8 = (13,  2,  8, 4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7, 
       1, 15, 13, 8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2, 
       7, 11,  4, 1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8, 
       2,  1, 14, 7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11)

# A list of all the secret boxes
sBoxes = (S1, S2, S3, S4, S5, S6, S7, S8)

When finding the replacement value in the sbox, the first and last bits are used to determine the row, and the middle four are used to determine the column. Then the value at that location is converted to binary and replaces the 6 bits.

For example, if the first chunk were ```001011```, the row would be ```01``` (1) and column ```0101``` (5). On a zero index matrix, this would be the 2nd row and 6th column of the s-box 1, which will result in ```2```. The ```2``` is converted to binary and replaces the ```001011``` -> ```0010```.

In [12]:
# Substitution Example

# Prints S Box 1
print("S-Box 1\n"+"-"*7)
for x in range(4):
  for y in range(16):
    print(S1[x*16+y], end=" ")
  print()

chunk = '001011'
# First and Last Bit
row = chunk[0]+chunk[-1]
# Middle 4 bits
column = chunk[1:-1]

print(f"\nOriginal Chunk: {chunk}")
print(f"Row: {row} ({int(row,2)}), Column: {column} ({int(column,2)})")
print(f"Replacement (Decimal): {S1[int(row,2)*16+int(column,2)]}")
print(f"After Subsitution: {chunk} -> {bin(S1[int(row,2)*16+int(column,2)])[2:].zfill(4)}")


S-Box 1
-------
14 4 13 1 2 15 11 8 3 10 6 12 5 9 0 7 
0 15 7 4 14 2 13 1 10 6 12 11 9 5 3 8 
4 1 14 8 13 6 2 11 15 12 9 7 3 10 5 0 
15 12 8 2 4 9 1 7 5 11 3 14 10 0 6 13 

Original Chunk: 001011
Row: 01 (1), Column: 0101 (5)
Replacement (Decimal): 2
After Subsitution: 001011 -> 0010


The entire 48-bit block goes through this process, where each chunk uses a different s-box to produce the 32-bit half.

In [13]:
def DataSubstitution(block):
  newBlock = ''
  for x in range(8):
    index = x*6
    row = int((block[index]+block[index+5]),2)
    col = int((block[index+1:index+5]),2)
    newBlock += bin(sBoxes[x][(16*row)+col])[2:].zfill(4)
  return newBlock

print(f"After XOR w/Key: {xorTemp} {len(xorTemp)} bits")
subTemp = DataSubstitution(xorTemp)
print(f"After Substitution: {subTemp} {len(subTemp)} bits")

After XOR w/Key: 111011110001010100011111101101101100111000110110 48 bits
After Substitution: 00001011110010010010110000001101 32 bits


## Permutation

Before the block is returned, a permutation is performed using the **Data Permutation** to shuffle the 32-bit block.

In [14]:
# Order used to Permutate the blocks
DataPermutation = (16,  7, 20, 21, 29, 12, 28, 17, 
                    1, 15, 23, 26,  5, 18, 31, 10, 
                    2,  8, 24, 14, 32, 27,  3,  9, 
                   19, 13, 30,  6, 22, 11,  4, 25)

print(f"After Substitution:\t{subTemp}")
print(f"After Permuation:\t{permute(subTemp, DataPermutation)}")

After Substitution:	00001011110010010010110000001101
After Permuation:	11011000000010010100100111101000


The full Feistel function:

In [15]:
def Feistel(data:str, key:str):
  data = permute(data, DataExpansion)
  data = XOR(data, key, 48)
  data = DataSubstitution(data)
  data = permute(data, DataPermutation)
  return data

# DES Round
---
The **DES Round** is the process of encrypting/decrypting a 64-bit block of data. It begins by splitting the block 64-bit block into 32-bit halves. Now for 16 rounds, the current right block becomes the new left block, and the new right becomes the XOR of the old left with the **Feistel** result of the old right and the round key.

$L_{n} = R_{n-1}$ <br />
$R_{n} = L_{n-1}$<span>&#8853;</span> $F({R_{n-1}, K})$

Once the two new halves are computed, the left is concatenated with the right and returned. When DES reaches the last round of encryption/decryption, the right is concatenated with the left and returned instead.

In [16]:
def DESRound(data:str, subkey:str, index:int):
  # Split the current block into 32-bit halves
  left = data[0:32]
  right = data[32:]

  # The new left because the old right
  lnew = right

  # The new right is the XOR of the old left and the Feistel(old right, subkey)
  rnew = XOR(left, Feistel(right, subkey), 32)

  # Return for final round
  if(index == 15):
    return rnew+lnew
  # Return for first 15 rounds
  return lnew+rnew

# Encryption
---

Encryption in DES takes the parts that we learned before and puts them all together. Encryption is responsible for generating the subkeys, transforming the plaintext into 64-bit blocks, and encrypting each block.

Before the 16 rounds of DES begin, data blocks are permutated with an **Initial Permutation**. After the rounds are completed, and the output is returned, it goes through a **Final Permutation**.

In [17]:
# Order used for initial shuffle of block before DES Rounds
DataInitialPermutation = (58, 50, 42, 34, 26, 18, 10, 2, 
                          60, 52, 44, 36, 28, 20, 12, 4, 
                          62, 54, 46, 38, 30, 22, 14, 6, 
                          64, 56, 48, 40, 32, 24, 16, 8, 
                          57, 49, 41, 33, 25, 17,  9, 1, 
                          59, 51, 43, 35, 27, 19, 11, 3, 
                          61, 53, 45, 37, 29, 21, 13, 5, 
                          63, 55, 47, 39, 31, 23, 15, 7)

# Order used for final shuffle of block after DES Rounds
DataFinalPermutation = (40, 8, 48, 16, 56, 24, 64, 32, 
                        39, 7, 47, 15, 55, 23, 63, 31, 
                        38, 6, 46, 14, 54, 22, 62, 30, 
                        37, 5, 45, 13, 53, 21, 61, 29, 
                        36, 4, 44, 12, 52, 20, 60, 28, 
                        35, 3, 43, 11, 51, 19, 59, 27, 
                        34, 2, 42, 10, 50, 18, 58, 26, 
                        33, 1, 41,  9, 49, 17, 57, 25)

## Full Encryption Function

In [18]:
def DESEncryption(plaintext:str, key:str):
  # First Generate the subkeys
  keys = keyScheduleGeneration(key)

  # Transfrom the plaintext into blocks
  dataBlocks = create64Blocks(plaintext)
  ciphertext = ''

  # Begin DES Rounds to encrypt
  for block in dataBlocks:

    # Initial Permutation
    block = permute(block, DataInitialPermutation)

    # Begin Rounds
    for b in range(16):
      block = DESRound(block, keys[b], b)
    
    # Final Permutation
    block = permute(block, DataFinalPermutation)

    # Add encrypted block to final ciphertext
    ciphertext += block
  return ciphertext


CIPHERTEXT = DESEncryption(PLAINTEXT, KEY)
print(f"Encrypted Message (Hex): {hex(int(CIPHERTEXT,2))[2:]}")
CIPHERTEXT = bin2ascii(CIPHERTEXT)
print(f"Encrypted Message (ASCII): {CIPHERTEXT}")

Encrypted Message (Hex): 6a0e79f552eda95b3de472d82d77c36a1971e077a951e2a98a286c85584dd35bc474ce3f555a42b2
Encrypted Message (ASCII): jyõRí©[=ärØ-wÃjqàw©Qâ©(lXMÓ[ÄtÎ?UZB²


# Decryption
---

Decryption is the same process as encryption for DES because it is a symmetric encryption algorithm. The only difference is that the keys are used in reverse, and the padding, if any, is removed.

In [19]:
# Unpadding Function
def unpad(ciphertext:str):
  padding = 8*[int(ciphertext[i:i+8],2) for i in range(0,len(ciphertext), 8)].count(0)
  return ciphertext[:-padding]

# DES Decryption
def DESDecryption(ciphertext:str, key:str):
  # First Generate the subkeys and then reverse them
  keys = keyScheduleGeneration(key)
  keys.reverse()
  
  # Transfrom the ciphertext into blocks
  dataBlocks = create64Blocks(ciphertext)

  ciphertext = ''

  # Begin DES Rounds to encrypt
  for block in dataBlocks:

    # Initial Permutation
    block = permute(block, DataInitialPermutation)

    # Begin Rounds
    for b in range(16):
      block = DESRound(block, keys[b], b)
    
    # Final Permutation
    block = permute(block, DataFinalPermutation)

    # Add encrypted block to final ciphertext
    ciphertext += block

  return unpad(ciphertext)

plaintext = DESDecryption(CIPHERTEXT, KEY)
print(f"Decrypted Message (Hex): {hex(int(plaintext,2))[2:]}")
plaintext = bin2ascii(plaintext)
print(f"Decrypted Message (ASCII): {plaintext}")

Decrypted Message (Hex): 444553206973207374726f6e6720627574206e6f7420636f6d706c696361746564
Decrypted Message (ASCII): DES is strong but not complicated


# Conclusion
---
You have now learned the inner working parts of the Data Encryption Standard (DES), including the Key Schedule Generation process, the Feistel Function operations, encryption, and decryption.

# Resources
1. https://en.wikipedia.org/wiki/Data_Encryption_Standard
1. https://www.commonlounge.com/discussion/5c7c2828bf6b4724b806a9013a5a4b99#detailed-description-of-key-scheduling-steps-with-examples
1. https://csrc.nist.gov/csrc/media/publications/fips/46/3/archive/1999-10-25/documents/fips46-3.pdf
1. https://www.geeksforgeeks.org/data-encryption-standard-des-set-1/
1. https://www.tutorialspoint.com/cryptography/data_encryption_standard.htm